In [104]:
pip install pyquery

In [105]:
pip install tenacity

In [121]:
pip install umls-api

  Created wheel for umls-api: filename=umls_api-0.1.0-py3-none-any.whl size=3408 sha256=706a6fb89f493b6bf7fdb15001abed2a50ddd3d4b1a9eabbd2ffb73da28ecb93
  Stored in directory: /root/.cache/pip/wheels/b1/0c/16/429473df1665cab9602747448f46d5fb701bf2e9634915ea71
Successfully built umls-api


In [5]:
import requests
import json
from lxml.html import fromstring
import pyquery
from __future__ import print_function
import pandas as pd
import umls_api

Authentication 

In [113]:
# uri="https://utslogin.nlm.nih.gov"
# auth_endpoint = "/cas/v1/api-key"

In [2]:
apikey = "e22b627f-226b-420b-9de2-935e10ff3350"
service="http://umlsks.nlm.nih.gov"

In [3]:
def get_granting_ticket():
     params = {'apikey': apikey}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post("https://utslogin.nlm.nih.gov/cas/v1/api-key",data=params,headers=h)
     response = fromstring(r.text)
     tgt = response.xpath('//form/@action')[0]
     return tgt

In [4]:
def get_single_ticket(tgt):
     params = {'service': service}
     h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
     r = requests.post(tgt,data=params,headers=h)
     st = r.text
     return st
   

In [6]:
def search_by_term(string):
  version = "2021AA"
  string = string
  tgt = get_granting_ticket()
  pageNumber=0

  while True:
    ##generate a new service ticket for each page if needed
    ticket = get_single_ticket(tgt)
    pageNumber += 1
    query = {'string':string,'ticket':ticket, 'pageNumber':pageNumber}
    r = requests.get("https://uts-ws.nlm.nih.gov/rest/search/"+version,params=query)
    r.encoding = 'utf-8'
    items  = json.loads(r.text)
    jsonData = items["result"]

    print("Results for page " + str(pageNumber)+"\n")
    
    ui_list = []
    
    for result in jsonData["results"]:
        
      try:
        print("ui: " + result["ui"])
        ui_list.append(result["ui"])
      except:
        NameError
      try:
        print("uri: " + result["uri"])
      except:
        NameError
      try:
        print("name: " + result["name"])
      except:
        NameError
      try:
        print("Source Vocabulary: " + result["rootSource"])
      except:
        NameError
      
      print("\n")
        
    
    ##Either our search returned nothing, or we're at the end
    if jsonData["results"][0]["ui"] == "NONE":
        break
    print("*********")

    return ui_list 
    

In [10]:
response = []
for i in search_by_term("multiple sclerosis"):
  r = umls_api.API(api_key = 'e22b627f-226b-420b-9de2-935e10ff3350').get_cui(cui = i)['result']['semanticTypes'][0]['name']
  response.append(r)

print(set(response))

Results for page 1

ui: C0026769
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0026769
name: Multiple Sclerosis
Source Vocabulary: MTH


ui: C0057378
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0057378
name: demyelinating factors
Source Vocabulary: MSH


ui: C0235962
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0235962
name: Multiple sclerosis aggravated
Source Vocabulary: MDR


ui: C0236169
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0236169
name: Multiple sclerosis-like syndrome
Source Vocabulary: MDR


ui: C0270784
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0270784
name: Multiple sclerosis of the brainstem
Source Vocabulary: SNOMEDCT_US


ui: C0338475
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0338475
name: Multiple sclerosis of the spinal cord
Source Vocabulary: SNOMEDCT_US


ui: C0393664
uri: https://uts-ws.nlm.nih.gov/rest/content/2021AA/CUI/C0393664
name: Multiple Sclerosis, Acute Relapsing
Source Vocab